# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп».  
Необходимо:  
- разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию  
- проверить корректность его работы
- качество не должно ухудшиться

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Категориальный признак "Пол" закодирован c помощью `1` и `0`.

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Пол,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
Возраст,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
Зарплата,5000.0,39916.3600,9900.083569,5300.0,33300.0,40200.0,46600.0,79000.0
Члены семьи,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
Страховые выплаты,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


Аномальных значений не обнаружено

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


В данных отсутствуют пропуски. Типы данных адекватны.

### Вывод
Файл открыт и изучен. В постобрабоке не нуждается.

## Умножение матриц

Признаки умножают на обратимую матрицу.  

**Вопрос:** Изменится ли качество линейной регрессии?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Ответ:** Не изменится.

**Обоснование:**  

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Общая формула:

$$
a = X \cdot (X^T \cdot X)^{-1} \cdot X^T \cdot y
$$

В общей формуле, умножим матрицу признаков X на обратимую матрицу P

$$
ap = X \cdot P \cdot
{({(X \cdot P)}^T  \cdot X \cdot P)}^{-1}
\cdot {(X \cdot P)}^T \cdot y
$$

Раскроим скобки
$$
ap = X \cdot P \cdot
{(P^{T} \cdot X^{T} \cdot X \cdot P)}^{-1} \cdot
P^{T} \cdot X^{T} \cdot y = 
$$
$$
X \cdot P \cdot P^{-1} \cdot
({{X}^T \cdot X)}^{-1} \cdot
{({P}^T)}^{-1} \cdot {P}^T \cdot {X}^T  \cdot y
$$

Воспользуемся свойством: $ P \cdot {P}^{-1} = E$ и ${P}^{-1} \cdot P = E$  

$E$ - единичная матрица, обладающая свойством: $A \cdot E = E \cdot A = A$.  

Таким образом: $ P \cdot P^{-1} = E $ и $ {(P^T)}^{-1} \cdot {P}^T = E$.

Формула преобретает вид:  
$$
ap = 
X \cdot
({{X}^T \cdot X)}^{-1} \cdot
{X}^T  \cdot y
$$  

Как мы можем видеть: $a = ap$

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
$$
w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$

### Вывод
При произведении матрицы признаков на обратимую матрицу, предсказания модели линейной регрессии  **не** изменяются.

## Алгоритм преобразования

**Алгоритм**

Данные приходят в закодированном виде. Разработаем алгоритм кодировки.  


- Генерируем случайную матрицу с помощью `numpy.random.random(size=None)`, потому что обратного преобразования задача не предпологает. В данной задаче подойдет этот метод.
- В качестве параметров для аргумента size передается кортеж вида (ширина признаков, шараина признаков). Пример: матрица признаков X размерами 10х5: size=(5,5).
- Создаем новую матрицу, путем перемножения матрицы признаков на матрицу, созданную с помощью `numpy.random.random()`

**Обоснование**

Обоснованием служит вывод из п.2.

## Проверка алгоритма

Для начала разобьем нашу выборку на обучающую и тестовую.

In [6]:
# датафрейм признаков
features = df.drop('Страховые выплаты', axis=1)
features.head(3)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0


In [7]:
# сериес целевых признаков
target = df['Страховые выплаты']
target.head()

0    0
1    1
2    0
3    0
4    0
Name: Страховые выплаты, dtype: int64

In [8]:
# разбивка на обучающие и тестовые выборки
train_features, test_features, train_target, test_target = train_test_split(
    features, target, test_size=0.2, random_state=20220421
)

In [9]:
# проверка размеров
print(train_features.shape)
print(train_target.shape)

print(test_features.shape)
print(test_target.shape)

(4000, 4)
(4000,)
(1000, 4)
(1000,)


In [10]:
# квадратная матрица случайных чисел
det = 0
P = 0

# проверка на обратимость через определитель матрицы
while det == 0:
    P = np.random.random(size=(features.shape[1], features.shape[1]))
    det = np.linalg.det(P)

def convert_feat(features):
    """функция кодирует признаки, путем умножения"""
    return np.dot(features, P)

Создадим закодированный выборки

In [11]:
# закодированная выборка признаков
# обучающая
train_P = convert_feat(train_features)

# тестовая
test_P = convert_feat(test_features)

### Проверка

Незакодированные данные

In [12]:
model = LinearRegression()

# обучаем модель
model.fit(train_features, train_target)

# предсказания
predictions = model.predict(test_features)

print(
    r2_score(test_target, predictions)
)

0.44194094621174695


По закодированным данным

In [13]:
model.fit(train_P, train_target)

predictions = model.predict(test_P)

print(
    r2_score(test_target, predictions)
)

0.44194094622678903


### Вывод
Проверка работы алгоритма прошла успешна. Модель обучается и предсказывает как на закодированнных так и незакодированных признаках.

## ВЫВОД
В работе было:
- открыт и изучен файл
- рассмотрено влияние умножения матрицы признаков на обратимую матрицу
- предложен алгоритм кодирования для защиты личных данных
- проверена работа алгоритма
- даны краткие выводы по каждому пункту